In [ ]:
suppressPackageStartupMessages({
    library(fgsea)
    library(data.table)
    library(ggplot2)
    library(factoextra)
    library(tidyverse)
    library(grid)
    library(shadowtext)
    library(gridExtra)
    library(lattice)
    library(tictoc)
    
    
    
    # maybe still needs:
    library(readr)
    library(scales)
    library(RColorBrewer)
    library(stringr)
    library(matrixStats)
    library(pheatmap)
    library(dplyr)
    library(ggrepel)
    library(Matrix)
    library(cowplot)
    library(tidyr)
    
    
    
    # probably doesn't need:
    # library(hrbrthemes)
    # library(tm)
    # library(proustr)
    # library(AnnotationHub)
#     library(forcats)
#     library(goseq)
    #     library(VennDiagram)
#     library(geneLenDataBase)
#     library(clusterProfiler))
#     library(enrichplot)
#     library(DOSE)
#     library(org.Hs.eg.db)
#     library(ggpubr)
#     library(scater)
#     library(future)
#     library(ggthemes)
#     library(patchwork)
#     library(Seurat)
#     library(limma)
#      library(Signac)
#     library(DESeq2)
#     library(EnsDb.Hsapiens.v86)
#     library(tximport)
# library(TxDb.Hsapiens.UCSC.hg38.knownGene)
# library(tidyverse)    
})
set.seed(42)

# Input Files:

In [ ]:
NUM_PATHWAYS_TO_PLOT = 20 # number of pathways to plot for each up or down 

#### Gene Set List

In [ ]:
pathwaysList = c('KEGG', 'GOMF', 'GOBP', 'GOCC', 'GO', 'REACTOME', 'ALL_PATHWAYS', 'HALLMARK_GENES')

#### Gene List Files:

In [ ]:
gene_list_files_dir = '/nfs/lab/relgamal/MAGMA/'
go_file <- paste0(gene_list_files_dir, 'c5.go.v2023.1.Hs.symbols.gmt')
go_mf_file <- paste0(gene_list_files_dir, 'c5.go.mf.v2023.1.Hs.symbols.gmt.txt')
go_bp_file <- paste0(gene_list_files_dir, 'c5.go.bp.v2023.1.Hs.symbols.gmt.txt')
go_cc_file <- paste0(gene_list_files_dir, 'c5.go.cc.v2023.1.Hs.symbols.gmt.txt')
kegg_file <- paste0(gene_list_files_dir, 'c2.cp.kegg.v2023.1.Hs.symbols.gmt')
reactome_file <- paste0(gene_list_files_dir, 'c2.cp.reactome.v2023.1.Hs.symbols.gmt')
hallmark_genes_file <- paste0(gene_list_files_dir, 'h.all.v2023.1.Hs.symbols.gmt.txt')


ribosomeProteinListFile_1 <- '/nfs/lab/relgamal/MAGMA/group-646.csv'
ribosomeProteinListFile_2 <- '/nfs/lab/relgamal/MAGMA/group-728.csv'
mitoProteinListFile <- '/nfs/lab/relgamal/MAGMA/group-729.csv'



#### Celltypes


In [ ]:

celltypes_noEndoC = c('acinar', 'all_pseudobulk' , 'alpha' , 'alpha_1' , 'alpha_2' , 'alpha_3' , 'alpha_4' , 
              'beta' , 'beta_1' , 'beta_2' , 'bulk38' , 'delta' , 'ductal' , 'gamma' , 'stellate')

celltypes = c(celltypes_noEndoC, 'endoC_treatment_time_24', 'endoC_treatment_time_72', 'endoC_timeTreatmentBase_withTimeCoV' )


#### Input Files

In [ ]:

indir_allCelltypes =  '/nfs/lab/projects/islet_multiomics_stress_CoCl2/pipeline/DESeq/output_files/'
fileSuffix = '.allGenesIncNonSig.DESeqResults.design_Donor_Treatment.CoCl2.fittype_Local.alpha_0.05.txt'

inputfilenames <- list()

for (celltype1 in celltypes_noEndoC){
    inputfilenames[[celltype1]] = paste0(indir_allCelltypes, celltype1, fileSuffix)
#     print(paste(celltype1, inputfilenames[[celltype1]]))
}



inputfilenames[['endoC_treatment_time_24']] = paste0(indir_allCelltypes, 
                                      'endoC_treatment_time_24.allGenesIncNonSig.DESeqResults.design_Donor_CoCl2_24.CoCl2.fittype_Local.alpha_0.05.txt')
inputfilenames[['endoC_timeTreatmentBase_withTimeCoV']] = paste0(indir_allCelltypes, 
                                      'endoC_timeTreatmentBase_withTimeCoV.allGenesIncNonSig.DESeqResults.design_Donor_TimeAsCov_Treatment.CoCl2.fittype_Local.alpha_0.05.txt')
inputfilenames[['endoC_treatment_time_72']] = paste0(indir_allCelltypes, 
                                      'endoC_treatment_time_72.allGenesIncNonSig.DESeqResults.design_Donor_CoCl2_72.CoCl2.fittype_Local.alpha_0.05.txt')



#### Output Dir 

In [ ]:
out_dir = '/nfs/lab/projects/islet_multiomics_stress_CoCl2/pipeline/GSEA/'

# Read in Pathways, etc 

In [ ]:
tic()
go <- gmtPathways(go_file)
go_mf <- gmtPathways(go_mf_file)
go_bp <- gmtPathways(go_bp_file)
go_cc <- gmtPathways(go_cc_file)
kegg <- gmtPathways(kegg_file)
reactome <- gmtPathways(reactome_file)
hallmark_genes <-  gmtPathways(hallmark_genes_file)

pathways <- c(go, kegg, reactome)
pathways2 <- c(go_bp,kegg,reactome)
all_go <- c(go_mf, go_bp, go_cc)
all_pathways <- c(go_mf, go_bp, go_cc, kegg, reactome, hallmark_genes)
toc()


pathways_Obj_List = list()
pathways_Obj_List[['GO']] = go
pathways_Obj_List[['GOMF']] = go_mf
pathways_Obj_List[['GOBP']] = go_bp
pathways_Obj_List[['GOCC']] = go_cc
pathways_Obj_List[['KEGG']] = kegg
pathways_Obj_List[['REACTOME']] = reactome
pathways_Obj_List[['ALL_PATHWAYS']] = all_pathways
pathways_Obj_List[['HALLMARK_GENES']] = hallmark_genes


In [ ]:

rpl <- fread(ribosomeProteinListFile_1, fill=TRUE, header=TRUE)
rps <- fread(ribosomeProteinListFile_2, fill=TRUE, header=TRUE)
mtr <- fread(mitoProteinListFile, fill=TRUE, header=TRUE)

ribo_proteins <- c(rpl$V2, rps$V2, mtr$V2)
ribo_proteins <- ribo_proteins[which(ribo_proteins != 'Approved symbol')]

# Read in DESeq Results

#### Set up List of Lists for whole notebook

In [ ]:
# List of List Structure:
# celltype    
    # DESeq results = 'DEGs'
    # DESes results, no Ribosome Genes = 'DEGs_noRib'
    # Gene Ranks = 'Ranks'
    #  pathway type  
            # GSEA results = 'Results'
            # top up = 'TopUp'
            # top down = 'TopDown'
    
resultsListOfLists = list()
for (celltype1 in celltypes){
#     print(celltype1)
    resultsListOfLists[[celltype1]] = list()
    resultsListOfLists[[celltype1]][['KEGG']] = list()
    resultsListOfLists[[celltype1]][['GOMF']] = list()
    resultsListOfLists[[celltype1]][['GOBP']] = list()
    resultsListOfLists[[celltype1]][['GOCC']] = list()
    resultsListOfLists[[celltype1]][['GO']] = list()
    resultsListOfLists[[celltype1]][['REACTOME']] = list()
    resultsListOfLists[[celltype1]][['ALL_PATHWAYS']] = list()
        resultsListOfLists[[celltype1]][['HALLMARK_GENES']] = list()
}

##### Read in Deseq results

In [ ]:
tic()

for (celltype1 in celltypes_noEndoC){
    # DESeq_Cell/Treatment.CoCl2.acinarfittypeLocal.cocl2.txt
#     print(celltype1)
    prefix = paste0(indir_allCelltypes)
    celltypefile = paste0(indir_allCelltypes, celltype1, fileSuffix)
    resultsListOfLists[[celltype1]][['DEGs']] = read.table(inputfilenames[[celltype1]])
}

resultsListOfLists[['endoC_treatment_time_24']][['DEGs']] = read.table(inputfilenames[['endoC_treatment_time_24']] )
resultsListOfLists[['endoC_timeTreatmentBase_withTimeCoV']][['DEGs']] = read.table(inputfilenames[['endoC_timeTreatmentBase_withTimeCoV']])
resultsListOfLists[['endoC_treatment_time_72']][['DEGs']] = read.table(inputfilenames[['endoC_treatment_time_72']])

toc()

#### Filter out ribosomal genes

In [ ]:
for (celltype1 in celltypes){
    degs <- resultsListOfLists[[celltype1]][['DEGs']]
    degs <- degs[which(!rownames(degs) %in% ribo_proteins),] 
    resultsListOfLists[[celltype1]][['DEGs_noRib']]  <- degs
}


#### Make Ranks Table

In [ ]:
get_DEG_rank_Table <- function (degsA){
    degs_rankA <- -log10(degsA$pvalue) * sign(degsA$log2FoldChange)
    names(degs_rankA) <- rownames(degsA)
    return (degs_rankA)
}

for (celltype1 in celltypes){
     resultsListOfLists[[celltype1]][['Ranks']] <- get_DEG_rank_Table (resultsListOfLists[[celltype1]][['DEGs_noRib']])
}

# Run GSEA

#### Functions

In [ ]:

get_GSEA_Results <- function (ranks, pathwaysListA){
    res <- fgsea(pathways = pathwaysListA, 
                  stats    = ranks,
                  #nPermSimple = 100000,
                  eps      = 0.0,
                  minSize  = 0,
                  maxSize  = 999)
    return (res)
}




processUpDownTable <- function (tableA1, sizeCutoff){
    
    tableA1 <- tableA1[which(tableA1$size <= sizeCutoff),]
    tableA1 <- tableA1[order(tableA1$pval),]
    tableA1 <- tableA1[1:NUM_PATHWAYS_TO_PLOT,]
    tableA1$LOG10P <- -log10(tableA1$pval)
    tableA1$pathway <- factor(tableA1$pathway, levels= tableA1$pathway)
    tableA1$pathway <- fct_rev(tableA1$pathway)
    return(tableA1)
}


filter_GSEA_Results <- function (res, sizeCutoff){
    resList = list()
    resUp <- res[which(res$ES > 0),]
    resDown <- res[which(res$ES < 0),]
    resList[['topUp']] = processUpDownTable(resUp, sizeCutoff)
    resList[['topDown']] = processUpDownTable(resDown, sizeCutoff)
    
    
    return(resList)
}


filter_GSEA_Results_v2 <- function (res, sizeCutoff){
    resList = list()
    resUp <- res[which(res$NES > 0),]
    resDown <- res[which(res$NES < 0),]
    resUp <- resUp[order(resUp$padj),]
    resDown <- resDown[order(resDown$padj),]
    resList[['topUp']] = processUpDownTable(resUp, sizeCutoff)
    resList[['topDown']] = processUpDownTable(resDown, sizeCutoff)
    
    
    return(resList)
}


## Get GSEA Results - non-EndoC

In [ ]:
# This is shorter than the next cell, but somehow wrong and created a weird atomic vector error when i ran it and had to restart the whole notebook

# # go <- gmtPathways('/nfs/lab/relgamal/MAGMA/c5.go.v2023.1.Hs.symbols.gmt')
# # go_mf <- gmtPathways('/nfs/lab/relgamal/MAGMA/c5.go.mf.v2023.1.Hs.symbols.gmt.txt')
# # go_bp <- gmtPathways('/nfs/lab/relgamal/MAGMA/c5.go.bp.v2023.1.Hs.symbols.gmt.txt')
# # go_cc <- gmtPathways('/nfs/lab/relgamal/MAGMA/c5.go.cc.v2023.1.Hs.symbols.gmt.txt')
# # kegg <- gmtPathways('/nfs/lab/relgamal/MAGMA/c2.cp.kegg.v2023.1.Hs.symbols.gmt')
# # reactome <- gmtPathways('/nfs/lab/relgamal/MAGMA/c2.cp.reactome.v2023.1.Hs.symbols.gmt')
# #     resultsListOfLists[[celltype1]][['KEGG']] = list()
# #     resultsListOfLists[[celltype1]][['GOMF']] = list()
# #     resultsListOfLists[[celltype1]][['GOBP']] = list()
# #     resultsListOfLists[[celltype1]][['GOCC']] = list()
# #     resultsListOfLists[[celltype1]][['GO']] = list()
# #     resultsListOfLists[[celltype1]][['REACTOME']] = list()pathways_Obj_Lists[['ALL_PATHWAYS']] 
# tic('OVERALL')
# for (celltype1 in celltypes_noEndoC){
#     print(celltype1)
#     degs_rank <- resultsListOfLists[[celltype1]][['Ranks']]
#     for (pathway1 in pathwaysList){
#         print(paste(celltype1,pathway1))
#         pathwayObj <- pathways_Obj_List[[pathway1]]
#         print(head(names(pathwayObj)))
#         tic(paste(celltype1, ' ', pathway1))
#         res <- get_GSEA_Results (degs_rank, pathwayObj )
#         resultsListOfLists[[celltype1]][[pathway1]][['results']] <- res
#         toc()
#     }
#     print('\n')
# }
# toc()

In [ ]:

tic('OVERALL')
for (celltype1 in celltypes_noEndoC){
    print(celltype1)
    degs_rank <- resultsListOfLists[[celltype1]][['Ranks']]
    print(head(degs_rank))
    ## REACTOME -- reactome
    tic(paste(celltype1, ' KEGG'))
    res <- get_GSEA_Results (degs_rank, kegg)
    resultsListOfLists[[celltype1]][['KEGG']][['results']] <- res
    toc()
    
   
    # GO --- go
    tic(paste(celltype1, ' GO'))
    res <- get_GSEA_Results (degs_rank, go)
    resultsListOfLists[[celltype1]][['GO']][['results']] <- res
    toc()
    
    # GOMF -- go_mf
    tic(paste(celltype1, ' GOMF'))
    res <- get_GSEA_Results (degs_rank, go_mf)
    resultsListOfLists[[celltype1]][['GOMF']][['results']] <- res
    toc()
    
    ## GOBP -- go_bp
    tic(paste(celltype1, ' GOBP'))
    res <- get_GSEA_Results (degs_rank, go_bp)
    resultsListOfLists[[celltype1]][['GOBP']][['results']] <- res
    toc()
    
     ## GOCC -- go_cc
    tic(paste(celltype1, ' GOCC'))
    res <- get_GSEA_Results (degs_rank, go_cc)
    resultsListOfLists[[celltype1]][['GOCC']][['results']] <- res
    toc()
    
    ## REACTOME -- reactome
    tic(paste(celltype1, ' REACTOME'))
    res <- get_GSEA_Results (degs_rank, reactome)
    resultsListOfLists[[celltype1]][['REACTOME']][['results']] <- res
    toc()
    
    
    ## REACTOME -- reactome
    tic(paste(celltype1, ' ALL_PATHWAYS'))
    res <- get_GSEA_Results (degs_rank, all_pathways)
    resultsListOfLists[[celltype1]][['ALL_PATHWAYS']][['results']] <- res
    toc()
    
    tic(paste(celltype1, 'HALLMARK_GENES'))
    res <- get_GSEA_Results (degs_rank, hallmark_genes)
    resultsListOfLists[[celltype1]][['HALLMARK_GENES']][['results']] <- res
    toc()
    
    
}
toc()

In [ ]:
head(resultsListOfLists[['beta_1']][['KEGG']][['results']])

### Write Tables to file

In [ ]:
tic()
for (celltype1 in celltypes_noEndoC){
    
    for (pathwayType in pathwaysList){
#         print(celltype1)
        res <- resultsListOfLists[[celltype1]][[pathwayType]][['results']]
        res$leadingEdge2 <- paste(as.character(res$leadingEdge))
        res$leadingEdge2 = gsub('\\"','',res$leadingEdge2)
        res$leadingEdge2 = gsub('c\\(','',res$leadingEdge2)
        res$leadingEdge2 = gsub('\\)','',res$leadingEdge2)
        res$leadingEdge2 = gsub(' ','',res$leadingEdge2)
        res$leadingEdge = NULL
#         print(head(res$leadingEdge2))
        outfilename = paste0(out_dir, 'GSEA_results_from_fGSEA.', celltype1, '.', pathwayType, '.results.txt' )
        
        
        write.table(res, 
            file= outfilename, 
            sep='\t', quote=FALSE, col.names = NA)
        
    }
}
toc()

In [ ]:
# head(kegg)

### Filter Results Tables

In [ ]:
sizeCutoff1 = 500
tic()
for (celltype1 in celltypes_noEndoC){
    
    for (pathwayType in pathwaysList){
#         print(celltype1)
        res <- resultsListOfLists[[celltype1]][[pathwayType]][['results']]
        resList1 <- filter_GSEA_Results (res, sizeCutoff1)
        resultsListOfLists[[celltype1]][[pathwayType]][['TopUp']] =  resList1[['topUp']] 
        resultsListOfLists[[celltype1]][[pathwayType]][['TopDown']] =  resList1[['topDown']] 
    }
}
toc()

# Re-label Pathways

#### relabel functions

In [ ]:
fixLabel <- function(nameB, pathwayNameB){
    path1B <- paste0(pathwayNameB, '_')
    newNameB = nameB
    newNameB = gsub(path1B, '', newNameB)
    newNameB = gsub('_', ' ', newNameB)
    newNameB = str_to_title(newNameB) 
    return (newNameB)
}

fixLabel_EndParens <- function(nameB, path1B, suffixReplaceB){
    newNameB = nameB
    newNameB = gsub(path1B, '', newNameB)
    newNameB = gsub('_', ' ', newNameB)
    newNameB = str_to_title(newNameB) 
    newNameB = paste0(newNameB, suffixReplaceB)
    return (newNameB)
}



makeFixLabelList_NoEndParens <- function(pathwayNameA, pathA){
    relabellistA <- list()
    go_namesA <- names(pathA)
    for (name1A in go_namesA){
        relabellistA[[name1A]] = fixLabel(name1A, pathwayNameA)
    }
    return(relabellistA)
}


makeFixLabelList_EndParens <- function(pathA){
    pathwayReplaceNameListA = c('GOBP_', 'GOMF_', 'GOCC_', 'KEGG_', 'REACTOME_', 'HALLMARK_')
    suffixReplaceList = list()
    suffixReplaceList[['GOBP_']] = ' (GO BP)'
    suffixReplaceList[['GOMF_']] = ' (GO MF)'
    suffixReplaceList[['GOCC_']] = ' (GO CC)'
    suffixReplaceList[['KEGG_']] = ' (KEGG)'
    suffixReplaceList[['REACTOME_']] = ' (REACTOME)'
    suffixReplaceList[['HALLMARK_']] = ' (Hallmark Gene Set)'
    
    relabellistA <- list()
    go_namesA <- names(pathA)
    for (name1A in go_namesA){
        newName1A = name1A
        for (pathReplaceA in pathwayReplaceNameListA){
            replaceSuffix = suffixReplaceList[[pathReplaceA]]
            if(startsWith(newName1A, pathReplaceA)   ){
               relabellistA[[name1A]] <- fixLabel_EndParens(newName1A, pathReplaceA, suffixReplaceList[[pathReplaceA]])
            }
        }
    }
    return(relabellistA)
}


fixTable_MakeLabelColumn <- function(tableA, replaceDFP1){
    
    # direct list lookup to list to column: 2.618 sec elapsed
    # check match, lookup list: 22.312 sec elapsed
    # match between dfs: 0.026 sec elapsed

    # tests for how long the relabelling takes:

    # 1. direct list lookup to list to column
    # res1xxxx = resultsListOfLists[['beta_1']][['ALL_PATHWAYS']][['results']]
    # replaceListPxxxxxx = pathwayRelabelLists[['ALL_PATHWAYS']] 
    # namesXXXXX <-  as.character(unlist(as.list(res1xxxx$pathway)))

    # tic('direct list lookup to list to column')
    # labs1xxxx <- list()
    # for (p in namesXXXXX){
    #     pReplacexxxx = replaceListPxxxxxx[[p]]
    #     if (length(pReplacexxxx) == 0){
    #         pReplacexxxx = p
    #     }
    #      labs1xxxx = append(labs1xxxx, pReplacexxxx)
    # }
    # res1xxxx$labelA <- labs1xxxx
    # toc()




    # 1. check match, lookup list
    # tic('check match, lookup list')
    # labs1xxxx <- list()
    # labs1 <- list()
    # for (p in namesXXXXX){
    #     pReplacexxxx = p
    #     for (l in names(replaceListPxxxxxx)){
    #          if ((p == l)){
    #              pReplacexxxx = replaceListPxxxxxx[[l]]

    #          }
    #     }
    #      labs1xxxx = append(labs1xxxx, pReplacexxxx)
    # }
    # res1xxxx$labelB <- labs1xxxx
    # toc()


    # # 3. match between dfs
    # tic('match between dfs')
    # replaceDF <- stack(replaceListPxxxxxx)
    # colnames(replaceDF) <- c('newLabel', 'origPathway')         
    # # replaceDF$origPathway <- as.character(replaceDF$origPathway)
    # # head(replaceDF)
    # res1xxxx$labelC <- replaceDF$newLabel[match(res1xxxx$pathway, replaceDF$origPathway)]
    # toc()
    # # head(res1xxxx,2)

    
    tableA$label2 <- replaceDFP1$newLabel[match(tableA$pathway, replaceDFP1$origPathway)]
    return (tableA)
}

#### make relabel lists

In [ ]:
tic()
pathwayRelabelLists = list()
# KEGG
pathwayRelabelLists[['KEGG']] <- makeFixLabelList_NoEndParens('KEGG', pathways_Obj_List[['KEGG']])
# GOMF
pathwayRelabelLists[['GOMF']] <- makeFixLabelList_NoEndParens('GOMF', pathways_Obj_List[['GOMF']])
# GOBP
pathwayRelabelLists[['GOBP']] <- makeFixLabelList_NoEndParens('GOBP', pathways_Obj_List[['GOBP']])
# GOCC
pathwayRelabelLists[['GOCC']] <- makeFixLabelList_NoEndParens('GOCC', pathways_Obj_List[['GOCC']])
# REACTOME
pathwayRelabelLists[['REACTOME']] <- makeFixLabelList_NoEndParens('REACTOME', pathways_Obj_List[['REACTOME']])
# GO
pathwayRelabelLists[['GO']]  <- makeFixLabelList_EndParens(pathways_Obj_List[['GO']])
# ALL_PATHWAYS
pathwayRelabelLists[['ALL_PATHWAYS']]  <- makeFixLabelList_EndParens(pathways_Obj_List[['ALL_PATHWAYS']])
# hallmark
pathwayRelabelLists[['HALLMARK_GENES']]  <- makeFixLabelList_EndParens(pathways_Obj_List[['HALLMARK_GENES']])
toc()


#### Make DFs of pathway re-label lists

In [ ]:
pathway_relabel_dfs_list = list()
tic()
for (pathwayType in pathwaysList){
    pathwayRelabelOb <- pathwayRelabelLists[[pathwayType]]
     replaceDF <- stack(pathwayRelabelOb)
    colnames(replaceDF) <- c('newLabel', 'origPathway')         
     pathway_relabel_dfs_list[[pathwayType]] <- replaceDF
}
toc()

##### relabel

In [ ]:
# bloop = c('alpha')
# blip = c('KEGG')
tic()
for (celltype1 in celltypes_noEndoC){
    
    for (pathwayType in pathwaysList){
        topUpTable = resultsListOfLists[[celltype1]][[pathwayType]][['TopUp']]
        topDownTable = resultsListOfLists[[celltype1]][[pathwayType]][['TopDown']]
        replaceDF = pathway_relabel_dfs_list[[pathwayType]] 
        topUpTable <- fixTable_MakeLabelColumn(topUpTable,   replaceDF)
        topDownTable <- fixTable_MakeLabelColumn(topDownTable,  replaceDF)
        topUpTable$label <- topUpTable$label2
        topDownTable$label <- topDownTable$label2
        topUpTable$label2 <- NULL
        topDownTable$label2 <- NULL

        resultsListOfLists[[celltype1]][[pathwayType]][['TopUp']] <- topUpTable
        resultsListOfLists[[celltype1]][[pathwayType]][['TopDown']] <- topDownTable
        
        
    }
}
toc()

In [ ]:
pathway_relabel_dfs_list[['KEGG']] 

In [ ]:
tic()
for (celltype1 in celltypes_noEndoC){
    for (pathwayType in pathwaysList){
#         print(paste(celltype1, pathwayType))
        res = resultsListOfLists[[celltype1]][[pathwayType]][['results']]
#         print(head(res))
        replaceDF = pathway_relabel_dfs_list[[pathwayType]] 
#         print(head(replaceDF))
        res <- fixTable_MakeLabelColumn(res,  replaceDF)
        res$label <- res$label2
#         print(head(res$label2))
        res$label2 <- NULL
        resultsListOfLists[[celltype1]][[pathwayType]][['Results_withLabel']] <- res
    }
}
toc()

#### Write Results to File


In [ ]:
tic()
for (celltype1 in celltypes_noEndoC){
    
    for (pathwayType in pathwaysList){
#         print(celltype1)
        res <- resultsListOfLists[[celltype1]][[pathwayType]][['Results_withLabel']]
        res$leadingEdge2 <- paste(as.character(res$leadingEdge))
        res$leadingEdge2 = gsub('\\"','',res$leadingEdge2)
        res$leadingEdge2 = gsub('c\\(','',res$leadingEdge2)
        res$leadingEdge2 = gsub('\\)','',res$leadingEdge2)
        res$leadingEdge2 = gsub(' ','',res$leadingEdge2)
        res$leadingEdge = NULL
#         print(head(res$leadingEdge2))
        outfilename = paste0(out_dir, 'GSEA_results_from_fGSEA.', celltype1, '.', pathwayType, '.results.withLabel.txt' )
        
        
        write.table(res, 
            file= outfilename, 
            sep='\t', quote=FALSE, col.names = NA)
        
    }
}
toc()

In [ ]:
resultsListOfLists[['beta_1']][['HALLMARK_GENES']][['Results_withLabel']]

### Write results to rds

In [ ]:
tic()
for (celltype1 in celltypes_noEndoC){
    
    for (pathwayType in pathwaysList){
#         print(celltype1)
        res <- resultsListOfLists[[celltype1]][[pathwayType]][['Results_withLabel']]
#         res$leadingEdge2 <- paste(as.character(res$leadingEdge))
#         res$leadingEdge2 = gsub('\\"','',res$leadingEdge2)
#         res$leadingEdge2 = gsub('c\\(','',res$leadingEdge2)
#         res$leadingEdge2 = gsub('\\)','',res$leadingEdge2)
#         res$leadingEdge2 = gsub(' ','',res$leadingEdge2)
#         res$leadingEdge = NULL
#         print(head(res$leadingEdge2))
        outfilename = paste0(out_dir, 'GSEA_results_from_fGSEA.', celltype1, '.', pathwayType, '.results.withLabel.wGeneLists.rds' )
        
         saveRDS(res, file=outfilename)
        
    }
}
toc()